[View in Colaboratory](https://colab.research.google.com/github/tarosaku/sakutest/blob/master/stopsign_detection_test_new.ipynb)

Python2ならば動くが、Python3ではエラーになる

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install --user lxml
!pip install --user contextlib2

In [0]:
cd /content

In [0]:
!rm -r models
!git clone https://github.com/tensorflow/models.git

In [0]:
!rm -r cocoapi
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

In [0]:
cd /content/models/research

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
!python object_detection/builders/model_builder_test.py

In [0]:
# images.tar.gz,annotations.tar.gzをローカルからアップデートする場合
# show upload dialog (images.tar.gz,annotations.tar.gzを選択)
#from google.colab import files
#uploaded = files.upload()

In [0]:
# images.tar.gz,annotations.tar.gzをgoogledriveから読み込む場合
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# images.zip,annotations.zipをgoogledriveから読み込む（続き）
id = '1kM4myEkQ_4ecFTGwaWda7csycVac0MKt'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('images.zip')
id = '1wtU9-m1AWyoChnYDVT8Cb2ovsTUA6oC-'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('annotations.zip')

In [0]:
#!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
#!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!unzip images.zip
!unzip annotations.zip

In [0]:
rm faster_rcnn_resnet101_stopsign*

In [15]:
#3ファイルをアップ
#stopsign_label_map.pbtxt
#generate_tfrecord.py
#faster_rcnn_resnet101_stopsign.config
from google.colab import files
uploaded = files.upload()

Saving faster_rcnn_resnet101_stopsign.config to faster_rcnn_resnet101_stopsign.config


In [0]:
#xml_to_csv.py
#https://github.com/datitran/raccoon_dataset/blob/master/xml_to_csv.py
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations/xmls')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('stopsign_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()

In [21]:
!python generate_tfrecord.py --csv_input=stopsign_labels.csv  --output_path=train.record
!python generate_tfrecord.py --csv_input=stopsign_labels.csv  --output_path=eval.record

Successfully created the TFRecords: /content/models/research/train.record
Successfully created the TFRecords: /content/models/research/eval.record


In [0]:
#!python object_detection/dataset_tools/create_pet_tf_record.py \
#    --label_map_path=object_detection/data/pet_label_map.pbtxt \
#    --data_dir=`pwd` \
#    --output_dir=`pwd`

In [0]:
!ls -l *record*


In [0]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* .

In [0]:
#!cp object_detection/samples/configs/faster_rcnn_resnet101_pets.config .
#!sed -i "s|PATH_TO_BE_CONFIGURED|/content/models/research|g" faster_rcnn_resnet101_pets.config
#!sed -i "s|/content/models/research/pet_label_map.pbtxt|/content/models/research/object_detection/data/pet_label_map.pbtxt|g" faster_rcnn_resnet101_pets.config
#!sed -i "s|/content/models/research/pet_train.record|/content/models/research/pet_train_with_masks.record|g" faster_rcnn_resnet101_pets.config
#!sed -i "s|/content/models/research/pet_val.record|/content/models/research/pet_val_with_masks.record|g" faster_rcnn_resnet101_pets.config

In [0]:
#!sed -i "s|PATH_TO_BE_CONFIGURED|/content/models/research|g" faster_rcnn_resnet101_pets.config

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
get_ipython().system_raw('tensorboard --logdir /content/models/research --host 0.0.0.0 --port 6006 &')

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!mkdir train

In [0]:
# python3対応
# object_detection_evaluation.pyのprintをpython3対応する修正が必要（括弧を追加）。ローカルで修正してアップする
# show upload dialog

#from google.colab import files
#uploaded = files.upload() #修正済のobject_detection_evaluation.pyをアップロード
#
#!mv object_detection_evaluation.py /content/models/research/object_detection/utils/object_detection_evaluation.py

In [0]:
# python3対応
# /content/models/research/object_detection/model_lib.pyの「itervalues」を「items」に置換。ローカルで修正してアップする
# show upload dialog

#from google.colab import files
#uploaded = files.upload() #修正済のmodel_lib.pyのをアップロード
#
#!mv model_lib.py /content/models/research/object_detection/model_lib.py

In [0]:
# エラーになる　何が悪いのか？　by sakurai 20180717
# python2ならば動いたが 10分程度経過するとwarningが出る by sakurai 20180720
# 「Ignoring detection with image id XXXX since it was previously added" & "Ignoring ground truth with image id XXXX since it was previously added"」
# 以下と同じ状況か？
# https://stackoverflow.com/questions/51407435/cloud-ml-engine-warning-message-on-object-detection-training-ignoring-ground-t/51419239
# 試しに下のパラメータを「--num_eval_steps=1101」にしたら、warningが消えて進み始めたみたい
#!python object_detection/model_main.py \
#    --pipeline_config_path=/content/models/research/faster_rcnn_resnet101_stopsign.config \
#    --model_dir=/content/models/research/train \
#    --num_train_steps=50000 \
#    --num_eval_steps=2000 \
#    --alsologtostderr

In [0]:
get_ipython().system_raw('python object_detection/model_main.py \
    --pipeline_config_path=/content/models/research/faster_rcnn_resnet101_stopsign.config \
    --model_dir=/content/models/research/train \
    --num_train_steps=50000 \
    --num_eval_steps=100 \
    --alsologtostderr 2>&1 /content/log/tb.log &')

In [37]:
!ls -l train

total 475068
-rw-r--r-- 1 root root        81 Aug 13 07:57 checkpoint
-rw-r--r-- 1 root root  26058004 Aug 13 07:57 events.out.tfevents.1534147017.4f8391bf7f78
-rw-r--r-- 1 root root  14267686 Aug 13 07:57 graph.pbtxt
-rw-r--r-- 1 root root 438518392 Aug 13 07:57 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root     40511 Aug 13 07:57 model.ckpt-0.index
-rw-r--r-- 1 root root   7567473 Aug 13 07:57 model.ckpt-0.meta
-rw-r--r-- 1 root root      3204 Aug 13 07:56 pipeline.config


In [36]:
!ps -ef

UID        PID  PPID  C STIME TTY          TIME CMD
root         1     0  0 06:44 ?        00:00:00 /bin/bash -e /datalab/run.sh
root        64     1  0 06:45 ?        00:00:00 node /tools/node/bin/forever --m
root        74    64  0 06:45 ?        00:00:01 /tools/node/bin/node /datalab/we
root        80    74  0 06:45 ?        00:00:02 /usr/bin/python2 /usr/local/bin/
root        88    80  0 07:42 ?        00:00:07 /usr/bin/python -m ipykernel_lau
root       572     1  5 07:55 ?        00:00:05 /usr/bin/python2 /usr/local/bin/
root       579     1  0 07:56 ?        00:00:00 ./ngrok http 6006
root       603     1 97 07:56 ?        00:00:58 python2 object_detection/model_m
root      5376    88 99 07:57 pts/0    00:00:01 /bin/sh -c ps -ef
root      5377  5376  0 07:57 pts/0    00:00:00 ps -ef


In [0]:
!kill 5051 #model_main.pyをkillする

In [0]:
!rm -r  exported_graphs; mkdir exported_graphs

In [0]:
!export CHECKPOINT_NUMBER=6395; python object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path faster_rcnn_resnet101_stopsign.config \
    --trained_checkpoint_prefix train/model.ckpt-${CHECKPOINT_NUMBER} \
    --output_directory exported_graphs

In [0]:
!zip -r exp_g.zip exported_graphs

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'exp_g.zip'})
uploaded.SetContentFile('exp_g.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
import tensorflow as tf
import re


for event in tf.train.summary_iterator('train/events.out.tfevents.1531795962.b2fc742a6800'):
    for value in event.summary.value:
        if value.tag == 'Losses/Loss/RPNLoss/objectness_loss':
            if value.HasField('simple_value'):
                print(value.simple_value)

In [0]:
!uptime #know how long you have to use this instance

In [0]:
!python test1.py Pets.jpg

In [0]:
#!rm test1.py
!wget http://tegacaypharmacy.com/wp-content/uploads/2016/02/Pets.jpg

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from IPython.display import Image
Image("out.png", width=640, height=480)

In [0]:
!grep -4 32 object_detection/data/pet_label_map.pbtxt